<a href="https://colab.research.google.com/github/Bagito999/Deep-Learning/blob/main/Training_and_Deploying_TensorFlow_Models_at_Scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Persiapan

In [13]:
# Aktifkan TensorFlow 2.x hanya jika di Colab
try:
    import google.colab
    %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

# Cek dan atur konfigurasi GPU agar tidak mengalokasikan seluruh VRAM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth GPU diaktifkan.")
    except RuntimeError as e:
        print("Error saat mengatur memory growth:", e)
else:
    print("GPU tidak terdeteksi.")

# Tampilkan daftar GPU yang terdeteksi
print("GPUs:", gpus)


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Error saat mengatur memory growth: Physical devices cannot be modified after being initialized
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


#Export dan Serving dengan SavedModel & TF‑Serving

##Save/Load a SavedModel

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import os
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import numpy as np

# Buat folder model jika belum ada
model_dir = "my_model"
os.makedirs(model_dir, exist_ok=True)

# Load dataset Iris
iris = load_iris()
X, y = iris.data.astype("float32"), iris.target

# Split data (train dan validation)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Bangun model Sequential sederhana
model = keras.Sequential([
    layers.Input(shape=(4,)),              # Input: 4 fitur
    layers.Dense(16, activation="relu"),   # Hidden layer 1
    layers.Dense(16, activation="relu"),   # Hidden layer 2
    layers.Dense(3, activation="softmax")  # Output: 3 kelas
])

# Compile model
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Latih model
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=8,
    verbose=2
)

# Simpan model ke format SavedModel
tf.saved_model.save(model, model_dir)
print(f"SavedModel berhasil disimpan ke folder: {model_dir}/")


Epoch 1/10
15/15 - 5s - 357ms/step - accuracy: 0.3333 - loss: 3.4625 - val_accuracy: 0.3333 - val_loss: 2.8502
Epoch 2/10
15/15 - 1s - 67ms/step - accuracy: 0.3333 - loss: 2.2331 - val_accuracy: 0.3333 - val_loss: 1.7511
Epoch 3/10
15/15 - 0s - 9ms/step - accuracy: 0.1750 - loss: 1.4248 - val_accuracy: 0.0000e+00 - val_loss: 1.2119
Epoch 4/10
15/15 - 0s - 9ms/step - accuracy: 0.2833 - loss: 1.1418 - val_accuracy: 0.6333 - val_loss: 1.0904
Epoch 5/10
15/15 - 0s - 11ms/step - accuracy: 0.6083 - loss: 1.0724 - val_accuracy: 0.6667 - val_loss: 1.0378
Epoch 6/10
15/15 - 0s - 6ms/step - accuracy: 0.6250 - loss: 1.0232 - val_accuracy: 0.6667 - val_loss: 0.9835
Epoch 7/10
15/15 - 0s - 10ms/step - accuracy: 0.6250 - loss: 0.9748 - val_accuracy: 0.7000 - val_loss: 0.9333
Epoch 8/10
15/15 - 0s - 6ms/step - accuracy: 0.6750 - loss: 0.9270 - val_accuracy: 0.8667 - val_loss: 0.8850
Epoch 9/10
15/15 - 0s - 11ms/step - accuracy: 0.8667 - loss: 0.8787 - val_accuracy: 1.0000 - val_loss: 0.8366
Epoch 10/

##TensorFlow Serving

In [15]:
# Install Flask & flask-ngrok
!pip install -q flask flask-ngrok

In [22]:
from flask import Flask, request, jsonify
import tensorflow as tf
import numpy as np
import os

# Gunakan ngrok hanya jika tersedia dan diinginkan (opsional)
try:
    from flask_ngrok import run_with_ngrok
    USE_NGROK = True
except ImportError:
    USE_NGROK = False

# Inisialisasi Flask
app = Flask(__name__)
if USE_NGROK:
    run_with_ngrok(app)  # Untuk Colab atau uji coba publik

# Path ke SavedModel
MODEL_PATH = "my_model"

# Pastikan model tersedia
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Model path '{MODEL_PATH}' tidak ditemukan.")

# Load SavedModel
loaded_model = tf.saved_model.load(MODEL_PATH)
infer = loaded_model.signatures["serving_default"]

@app.route("/predict", methods=["POST"])
def predict():
    try:
        data = request.get_json(force=True)
        instances = np.array(data["instances"], dtype=np.float32)

        # Ubah ke tensor dan lakukan inferensi
        tf_inputs = tf.constant(instances)
        outputs = infer(tf_inputs)

        # Ambil key output pertama (biasanya 'output_0')
        out_key = list(outputs.keys())[0]
        preds = outputs[out_key].numpy().tolist()

        return jsonify({"predictions": preds})

    except Exception as e:
        return jsonify({"error": str(e)}), 400

# Jalankan server
if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information abo

#Convert ke TFLite untuk Mobile/Embedded

In [23]:
import tensorflow as tf
import os

# Path ke folder SavedModel
saved_model_dir = "my_model"

# Pastikan folder model ada
if not os.path.exists(saved_model_dir):
    raise FileNotFoundError(f"Folder model '{saved_model_dir}' tidak ditemukan.")

# Buat converter dari SavedModel (bukan Keras model)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)

# (Opsional) Quantization untuk mengurangi ukuran model
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Lakukan konversi
tflite_model = converter.convert()

# Simpan ke file .tflite
output_path = "model.tflite"
with open(output_path, "wb") as f:
    f.write(tflite_model)

print(f"Model berhasil dikonversi ke TFLite dan disimpan sebagai: {output_path}")
print(f"Ukuran model TFLite: {len(tflite_model)} bytes")


Model berhasil dikonversi ke TFLite dan disimpan sebagai: model.tflite
Ukuran model TFLite: 3844 bytes


#Menggunakan GPU

In [24]:
import tensorflow as tf
import numpy as np
import os

# Pengaturan GPU agar tidak langsung menggunakan seluruh VRAM
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"{len(gpus)} GPU terdeteksi. Memory growth diaktifkan.")
    except RuntimeError as e:
        print("Gagal mengatur memory growth:", e)
else:
    print("Tidak ada GPU terdeteksi.")

# Membangun model Keras sederhana
model = tf.keras.Sequential([
    tf.keras.layers.Dense(512, activation='relu', input_shape=(1000,)),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Kompilasi model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Membuat data dummy (10000 sampel, 1000 fitur)
X = np.random.rand(10000, 1000).astype('float32')
y = np.random.randint(0, 10, size=(10000,))

# Melatih model dan simpan hasil history
history = model.fit(
    X, y,
    epochs=3,
    batch_size=128,
    verbose=2
)

# Ambil nilai akurasi dan loss terakhir
final_loss = history.history['loss'][-1]
final_acc = history.history['accuracy'][-1]

# Tampilkan hasil akhir
print(f"\nHasil akhir training:")
print(f"Loss terakhir   : {final_loss:.4f}")
print(f"Akurasi terakhir: {final_acc:.4f}")


Gagal mengatur memory growth: Physical devices cannot be modified after being initialized


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


79/79 - 1s - 19ms/step - accuracy: 0.0965 - loss: 2.3771
Epoch 2/3


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


79/79 - 1s - 8ms/step - accuracy: 0.1118 - loss: 2.3027
Epoch 3/3
79/79 - 0s - 2ms/step - accuracy: 0.1305 - loss: 2.2896

Hasil akhir training:
Loss terakhir   : 2.2896
Akurasi terakhir: 0.1305


#Multi‑GPU dengan Distribution Strategies

In [25]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# Setup strategi distribusi (untuk multi-GPU jika tersedia)
gpus = tf.config.list_physical_devices('GPU')
if not gpus:
    print("⚠️ Tidak ada GPU terdeteksi. Model tetap bisa jalan di CPU.")
strategy = tf.distribute.MirroredStrategy()
print("Jumlah GPU aktif:", strategy.num_replicas_in_sync)

# Bangun model di dalam scope strategi
with strategy.scope():
    model = keras.Sequential([
        keras.layers.Dense(256, activation='relu', input_shape=(1000,)),
        keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

# Buat dummy data
X = np.random.rand(20000, 1000).astype('float32')
y = np.random.randint(0, 10, size=(20000,))

# Latih model dan simpan history
history = model.fit(
    X, y,
    epochs=5,
    batch_size=256,
    verbose=2
)

# Ambil nilai akurasi dan loss terakhir dari training
final_loss = history.history['loss'][-1]
final_acc = history.history['accuracy'][-1]

# Tampilkan hasil akhir
print("\nHasil akhir training:")
print(f"Loss terakhir   : {final_loss:.4f}")
print(f"Akurasi terakhir: {final_acc:.4f}")


Jumlah GPU aktif: 1


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that 

Epoch 1/5
79/79 - 1s - 12ms/step - accuracy: 0.1001 - loss: 2.3415
Epoch 2/5
79/79 - 1s - 7ms/step - accuracy: 0.1031 - loss: 2.3030
Epoch 3/5


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

79/79 - 0s - 6ms/step - accuracy: 0.1087 - loss: 2.3013
Epoch 4/5
79/79 - 0s - 5ms/step - accuracy: 0.1077 - loss: 2.3004
Epoch 5/5


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


79/79 - 0s - 6ms/step - accuracy: 0.1098 - loss: 2.3001

Hasil akhir training:
Loss terakhir   : 2.3001
Akurasi terakhir: 0.1098


#Black‑Box Hyperparameter Tuning dengan KerasTuner

In [27]:
# Install keras-tuner jika belum tersedia
!pip install -q keras-tuner

import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

# Optional: aktifkan GPU memory growth agar tidak boros
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
        except:
            pass

# Fungsi untuk membangun model
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.Dense(
        units=hp.Int('units', min_value=32, max_value=256, step=32),
        activation='relu',
        input_shape=(1000,)
    ))
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice('lr', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Inisialisasi RandomSearch tuner
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='tuner_dir',
    project_name='chapter19',
    overwrite=True  # overwrite hasil tuning sebelumnya jika ada
)

# Buat dummy dataset
X = np.random.rand(5000, 1000).astype('float32')
y = np.random.randint(0, 10, size=(5000,))

# Jalankan tuning
tuner.search(X, y, epochs=5, validation_split=0.2, verbose=2)

# Ambil model dan hyperparameter terbaik
best_model = tuner.get_best_models(num_models=1)[0]
best_hp = tuner.get_best_hyperparameters(1)[0]

# Tampilkan hasil terbaik
print("\n📌 Hyperparameter terbaik:")
print(f" - Learning rate : {best_hp.get('lr')}")
print(f" - Jumlah units  : {best_hp.get('units')}")


Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.10899999737739563

Best val_accuracy So Far: 0.12200000137090683
Total elapsed time: 00h 00m 35s

📌 Hyperparameter terbaik:
 - Learning rate : 0.0001
 - Jumlah units  : 128


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


#  Training and Deploying TensorFlow Models at Scale – Ringkasan

##  1. GPU Configuration
- TensorFlow mendeteksi GPU yang tersedia.
- Pengaturan `memory_growth` diaktifkan agar tidak mengambil seluruh VRAM secara langsung.
- Ini penting untuk efisiensi saat menggunakan multi-GPU atau resource terbatas.

##  2. Pembuatan Model
- Model `Sequential` dibangun dengan layer `Dense`.
- Input memiliki 1000 fitur dan output terdiri dari 10 kelas.
- Model dikompilasi menggunakan Adam Optimizer dan `sparse_categorical_crossentropy` untuk klasifikasi multi-kelas.

##  3. Training Awal
- Model dilatih dengan data dummy sebanyak 10.000–20.000 sampel.
- Akurasi dan loss akhir dicetak setelah proses training selesai.

##  4. Multi-GPU dengan MirroredStrategy
- `tf.distribute.MirroredStrategy()` digunakan untuk membagi beban training ke banyak GPU.
- Model dibuat di dalam `strategy.scope()` untuk sinkronisasi antar perangkat.
- Batch training otomatis dibagi sesuai jumlah GPU.

##  5. Hyperparameter Tuning dengan Keras Tuner
- `keras-tuner` digunakan untuk mencari kombinasi terbaik dari:
  - Jumlah unit layer tersembunyi (`units`: 32–256)
  - Learning rate (`lr`: 1e-2, 1e-3, 1e-4)
- Dilakukan pencarian acak (`RandomSearch`) sebanyak 5 percobaan (`trials`) dengan validasi 20%.
- Akurasi validasi (`val_accuracy`) digunakan sebagai metrik tujuan.

##  6. Hasil Tuning
- Best `val_accuracy`: **~12.2%**, menunjukkan bahwa model belum belajar secara optimal.
- Hyperparameter terbaik:
  - **Learning rate**: `0.0001`
  - **Units**: `128`
- Kemungkinan model underfitting karena data dummy bersifat acak, tidak bermakna secara klasifikasi.

---

##  Kesimpulan
- Semua pipeline pelatihan dan tuning telah berhasil dijalankan dalam skala besar menggunakan GPU dan strategi distribusi.
- Akurasi rendah dikarenakan data dummy yang tidak memiliki pola nyata.
- Pipeline ini siap untuk diadaptasi ke dataset riil untuk klasifikasi skala besar dan deployment ke model TFLite atau server Flask/REST API.